In [2]:
import os
import pandas as pd
import re
i = 0
filepaths = []
# "C:/Users/janja/Desktop/firstData"
for root, dirs, files in os.walk("C:/Users/janja/OneDrive/Pulpit/DaneMGR", topdown=True):
    for name in dirs:
        if (bool(re.findall('\d$', name)) == False):
            Path = (root + '/' + name)
            filepaths.append(re.sub('DaneMGR\\\\', 'DaneMGR/', Path))

In [3]:
from os import walk
filenameList = []
fullPath = []
for Path in filepaths:
    for (dirpath, dirnames, filenames) in walk(Path):
        for name in filenames:
            if (bool(re.findall('fast_Unknown', name)) == True) and name not in filenameList:
                NewName = re.sub('._CsvLog', 'CsvLog', name)
                filenameList.append(NewName)
                fullPath.append(Path + '/' + NewName)

In [4]:
T = 'N' #500000 #'N' # describes which rows multiplied by n should be taken into the dataset #If chosen parameter is N the rows will not get dropped
fields = ['Infinity|RESP.ONLY_ONE_IN_GROUP [OHM]', 'Infinity|SPO2.SPO2_PULSE [COUNTS]']
df_0 = pd.DataFrame()
df_1 = pd.DataFrame()

files_total = len(fullPath)
i = 1
for path in fullPath:
    print(path)
    df_local = pd.read_csv(path, sep = ',', encoding = 'UTF-8', usecols=fields)
    if T != 'N':
        df_local = df_local[df_local.index % T == 0] #Set to 2000 as 1 second is 20 observations
    match = re.findall("/B/Csv",path)
    if bool(match) == True:
        df_1 = df_1.append(df_local)
    else:
        df_0 = df_0.append(df_local)
    #neo = re.findall('\/([\d]{1,2})\/', path)
    #df_local.insert(0,'neonate', str(neo))
    #print(df_local)
    print(f"Imported file number: {i}, from files total: {files_total}, and that is {i*100/files_total:.2f}%")
    i+=1

C:/Users/janja/OneDrive/Pulpit/DaneMGR/1/B/CsvLogBase_2022-01-15_191202.371_fast_Unknown.csv
Imported file number: 1, from files total: 98, and that is 1.02%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/1/R/CsvLogBase_2022-01-15_235231.870_fast_Unknown.csv
Imported file number: 2, from files total: 98, and that is 2.04%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/10/B/CsvLogBase_2022-03-22_190208.416_fast_Unknown.csv
Imported file number: 3, from files total: 98, and that is 3.06%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/10/R/CsvLogBase_2022-03-22_145436.972_fast_Unknown.csv
Imported file number: 4, from files total: 98, and that is 4.08%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/11/B/CsvLogBase_2022-03-24_155741.211_fast_Unknown.csv
Imported file number: 5, from files total: 98, and that is 5.10%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/11/R/CsvLogBase_2022-03-23_192319.765_fast_Unknown.csv
Imported file number: 6, from files total: 98, and that is 6.12%
C:/Users/janja/OneDrive/Pulpit/DaneMGR/12/B/CsvL

In [ ]:
data = [df_0,df_1]

In [ ]:
for i in range(len(data)):
    data[i] = data[i].interpolate()

In [ ]:
for i in range(len(data)):
    data[i] = data[i].rename({'Infinity|RESP.ONLY_ONE_IN_GROUP [OHM]': 'RESP.ONLY_ONE_IN_GROUP[OHM]','Infinity|SPO2.SPO2_PULSE [COUNTS]': 'SPO2.SPO2_PULSE[COUNTS]'}, axis=1)
    print(f'Number of blank spaces for the {i} position: \n {data[i].isna().sum()}')
    print(data[i].dtypes.value_counts())
    print(data[i].describe(), end = '\n')

In [ ]:
import math
import numpy as np

def segmentation(pd_data, window_size):
    np_data = pd_data.to_numpy()
    nb_timestamps, nb_sensors = np_data.shape
    
    #window_size = 100 # Size of the data segments, earlier there was the value of 100
    timestamp_idx = 0 # Index along the timestamp dimension
    segment_idx = 0 # Index for the segment dimension
    
    nb_segments = int(math.floor(nb_timestamps/window_size))
    print(f'Starting segmentation with a window size of {window_size} resulting in {nb_segments} segments.')
    data_to_save = np.zeros((nb_segments,window_size,nb_sensors),dtype=np.float32)

    while segment_idx < nb_segments:
        data_to_save[segment_idx] = np_data[timestamp_idx:timestamp_idx+window_size,:]
        timestamp_idx += window_size
        segment_idx += 1
    return data_to_save

In [ ]:
position = []
for i in range(len(data)):
    #shape = np.array(data[i]).shape
    #print(shape)
    segmented = segmentation(data[i], 100)
    #print(segmented)
    position.append(segmented)